# Import libraries

In [16]:
from dotenv import load_dotenv
import os
from langchain_anthropic import ChatAnthropic
from langchain import hub
from langchain_community.document_loaders import ObsidianLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

# Set variables

In [17]:
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Load, split and store embeddings

In [18]:
loader = ObsidianLoader("/Users/danielmcateer/Library/Mobile Documents/iCloud~md~obsidian/Documents/Ideaverse")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-A0iYi***************************************Eexq. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# Retrieve and generate using the relevant notes from Obsidian

In [5]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatAnthropic(temperature=0.5, anthropic_api_key=anthropic_api_key, model_name="claude-3-opus-20240229")

# Run RAG chain using LangChain

In [9]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# Get a response from the RAG chain

In [11]:
output = {}
curr_key = None
for chunk in rag_chain_with_source.stream("What does Andrej Karpathy say about the Transformers architecture?"):
    for key in chunk:
        if key not in output:
            output[key] = chunk[key]
        else:
            output[key] += chunk[key]
        if key != curr_key:
            print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
        else:
            print(chunk[key], end="", flush=True)
        curr_key = key
print(output)





question: What does Andrej Karpathy say about the Transformers architecture?

context: [Document(page_content="that are pretty obvious that they don't do well to going back to the original you know tail the cognitive tape right you can look at all The places the human is clearly superior to the transformer and start to look out for architectures that might change that dynamic. You know if you said how many meaningful breakthroughs are we away from the AI scientists that can produce your reka moments at a pace faster than human scientists tend to. It doesn't feel like it's that many you know I would say probably more than zero but it's probably less than four. ([Time\xa01:45:00](https://share.snipd.com/snip/ed3e95a6-2600-4f78-abd6-e4225b3340d2))", metadata={'created': 1709001561.6959817, 'last_accessed': 1709158969.422937, 'last_modified': 1709001554.7333877, 'path': '/Users/danielmcateer/Library/Mobile Documents/iCloud~md~obsidian/Documents/Ideaverse/Readwise/Podcasts/Looking Up the 

# Cleanup

In [8]:
vectorstore.delete_collection()